The explanations and what I will apply is from and after reading ***A neural probabilistic language model paper*** *(Bengio, Y., et al, 2003)*

So we learned about n-grams language models, and how we can predict the next word given the $n$ previous words. And we implemented a **very simple** bigram model.  
Now, my next step was to understand word embeddings is how n-grams and statistical language modeling translated into neural networks and deep learning.

So in *(Bengio, Y., et al, 2003)*, the motivations they had is that it's difficult to model the joint probablity distribution of multiple consecutive words (random variables with **discrete** values.) An example would be having a sentence of 10 words where each one could have any value of a vocabulary of $100000$ words, so you have about $100000^{10}$ different possibilities or as they say "free parameters" for that sentence.

They also explained that modeling continuous variables is easier for generalization because the approximation function we want to learn is expected to **have** some smooth properties (neural networks would be a good example.) So the continuous space has a smoother space unlike discrete spaces where are more, idk, **discrete?** And they explain more about discrete spaces:  
***"For discrete spaces, the generalization structure is not as obvious: any change of these discrete variables may have a drastic impact on the value of the function to be estimated, and when the number of values that each discrete variable can take is large, most observed objects are almost maximally far from each other in hamming distance."***

**<u>Disclaimer:</u>**  
We are now going into the approach proposed and the mathematics and the design of it. Since I myself am trying to learn here, I won't be following the paper exactly here, meaning that I could:  
1.  Add a little bit more mathematics to give you and myself a better idea of what's happening.  
2.  Branch quickly in what I am writing to explain some other concepts that will help us understand what we have here better, which might be trivial to you depending on your level and knowledge so you can skip. But it won't be long anyway, this isn't an article (yet!)  

So previously, we learned that you can predict the next word $w_t$ given all previous ones:  
    $$P(w_t | w_{1:t-1}) \tag{1}$$  
    
But in n-grams, we wanted to approximate $(1)$ by using just the $n$ words before $w_t$ as follows:  
    $$P(w_t | w_{t-n+1:t-1}) \approx P(w_t | w_{1:t-1}) \tag{2}$$

Remember the joint probability equation, assuming there's a dependance between variables $x$, $y$, and $z$ (which in our case are words):
    $$P(x \cap y \cap z) = P(x, y, z) = P(z) P(y | z) P(x | z, y) \tag{3}$$
Which can be generalized to:
    $$P(x_i, x_{i-1}...x_{1}) = \prod_{k = 1}^n{P(x_k | x_1, x_2...x_{k-1})} \tag{4}$$

And now that we can approximate the probability of $w_t$, we can get the joint probability of a sentence or a sequence of words $W$ of length $T$:  

$$P(w_t w_{t-1}...w_1) = \prod_{k=1}^T{P(w_k | w_{k-n+1:k-1})} \tag{5}$$

This is the function we would want to maximize.

Side note for the joint probability in $(3)$: if we assumed there's no dependance between $x$, $y$, and $z$, we have:
    $$P(x, y, z) = P(x) P(y) P(z) \tag{6}$$

We are now want to use neural network to find a good function $f$ that approximates this joint probability distribution for a specific corpus. $f$ would have a set of trainable parameters $\Theta$.  
Now from above and $(2)$:  
$$P(w_t | w_{t-n+1:t-1}) \approx P(w_t | w_{1:t-1}) \newline
\approx f(w_t, w_{t-1}...w_{t-n+1};\Theta) \tag{7}$$

You can notice that $f$ uses $w_t$ and the n previous words $(w_{t-1}...w{t-n+1})$, just like we do in n-grams modelling.  
Now, from $(5)$ & $(7)$, we want to get the joint probability of the whole word sequence $W$ of word count $T$:
$$P(w_t w_{t-1}...w_1) = \prod_{k=1}^T{P(w_k | w_{k-n+1:k-1})} \newline
\approx \prod_{t = 1}^T{f(w_t, w_{t-1}...w_{t-n+1};\Theta)} \tag{8}$$  

<u>What are the parameters included in $\Theta$?</u>

The parameters we have would depend on the model's architecture, we have mainly 2 parts (according to the architecture proposed in the text.):
1. Assuming we would have a set of vocabulary $V$ of size $|V|$, we want to create a mapping (a lookup table) $C$ for each word $w_i$ to a vector $v_{w_i}$ whose dimensions $\in \mathbb{R}^m$. In other words: $C(w_i) = v_{w_i} \mid v_{w_i} \in \mathbb{R}^m$.
2. The probability function $g$ that takes the mapping $C$ of the input words and maps them to the probability distribution over the vocab $V$.
    $$f(w_t, w_{t-1}...w_{t-n+1};\Theta) = g(i, C(w_{t-1}...w_{t-n+1}))$$

So $g$ outputs a vector of $|V|$ dimensions ($\in \mathbb{R}^{|V|}$), where the $i_{th}$ dimension or element corresponds to the probability of $i_{th}$ word being the next token:
$$\hat{P}(w_t=i | w_{1:t-1})$$

Then $C$ would be an $|V| \times m$ matrix. You can also say that $C$ is the word embeddings as a term we know of today.

We now want to start definining the loss function, which will be equivalent to $(8)$:
$$l = \prod_{t = 1}^T{f(w_t, w_{t-1}...w_{t-n+1};\Theta)} \tag{9}$$

We can transform the product into a summation by taking the log of both sides:
$$log(l) = log(\prod_{t = 1}^T{f(w_t, w_{t-1}...w_{t-n+1};\Theta)}) \newline
= \sum_{t = 1}^T{log(f(w_t, w_{t-1}...w_{t-n+1};\Theta))} \tag{10}$$
**<u>Why do we take the log:</u>**  
* Multiplying a lot of probabilities would make the resulting value approach $0$, or a numerical underflow when running on a computer which can mess up the calculations and results.
* Likewise if we were multiplying large numbers would result in an even bigger result, eventually leading to an overflow.

What we reached in $(10)$ is what we call Log Likelihood. So, from $(10)$, our loss function would be:
$$L = \sum_{t = 1}^T{log(f(w_t, w_{t-1}...w_{t-n+1};\Theta))} \tag{11}$$

We can also add a Regularization function $R$ that regularizes the set of parameters $\Theta$, or a subset of $\Theta$ called $\Omega$:
$$L = \sum_{t = 1}^T{log(f(w_t, w_{t-1}...w_{t-n+1};\Theta))} + R(\Omega) \tag{12}$$

But I won't use the regularized term $R(\Omega)$ in this implementation, so we will use $(11)$ as our Loss Function for the implementation.

Now for the network's architecture:  
<p align="center">
    <img src="../images/model_architecture.jpeg" alt="Bengio, et al. 2003" width="600"/>
</p>  

The first layer is the embeddings layer, then we have just one hidden layer before the $Softmax$ function. They also describe that **we can optionally connect the embeddings layer directly to the softmax layer.**

**<u>What what are the parameters we have?</u>**  

$$\Theta = (b, d, W, U, H, C)$$

| Parameter | Size | Optional | Description |
|:----------:|:----------:|:----------:|:----------:|
| $C$    | $\|V\| \times m$   | No   | The lookup table for the vector representation of each word in the vocabulary $\|V\|$.   |
| $H$    | $h \times (n-1) m$  | No   | The weights matrix for the hidden layer.   |
| $U$    | $\|V\| \times h$  | No   | The weights matrix for the hidden layer output to the softmax layer.   |
| $b$    | $\|V\|$  | No   | The output layer's biases.   |
| $d$    | $h$  | No   | The hidden layer's biases.   |
| $W$    | $\|V\| \times (n-1)m$  | Yes   | The weights matrix for optional connection between the embeddings layer and softmax layer.   |

Such that the logits of the output layer y that's passed to the softmax activation:
$$y = b + Wx + U tanh(d + Hx)$$

The softmax activation (predicted probability of word $w_t$ given the previous $n$):
$$\hat{P}(w_t | w_{t-1:t-n+1}) = \frac{\textit{e}^{y_{w_t}}}{\sum_{i}{\textit{e}^{y_i}}}$$


Perhaps that's enough reading, lets try to make a naive implementation of the architecture above with ***PyTorch***.  
And by ***naive*** I mean it's my first time building a Torch module that deals with sequences, so I probably won't do it the most efficient way.

# Neural Language Model

In [308]:
from torch import nn


# Our Neural Language Model class
class NLM(nn.Module):
    def __init__(self, v: int, m: int, n: int, h: int):
        super().__init__()
        self.v: int = v  #   Vocabulary size
        self.m: int = m  #   Embeddings dimensions
        self.n: int = n  #   Context size (the n-grams), we will actually input n-1 words to predict the n^th
        self.h: int = h  #   Hidden layer units (neurons)
        self.padding_token_idx: int = v  #   Indicates the index of the special token to be used to pad the a sequence of length < the context size

        #   I started with a randn tensor of shape (v, m) but turns out that Torch has a class for embeddings specifically,
        #   where this class stores the lookup table with numerical indices, so we still have to map the words to indices before entering the class
        #   or tokenization in other means
        self.c = nn.Embedding(
            num_embeddings=v + 1, embedding_dim=m, padding_idx=self.padding_token_idx
        )

        #   This way, the linear/Activation layer will take flattened embeddings and produce v outputs
        #   I did it this way as per the paper which says that x (output of the embeddings) would be just the concatenation of all feature vectors
        #   But you can preserve the dimensionality of the of having each embedding vector seperately even with Linear layers
        #   by making the in_features=embedding dimensions
        self.linear1 = nn.Linear(in_features=m * (n - 1), out_features=h)
        self.tanh = nn.Tanh()

        #   The output layer, linear to create a logits and softmax activation to normalize them to probabilities
        self.linear2 = nn.Linear(
            in_features=h, out_features=v + 1
        )  #   + 1 for the padding token
        self.softmax = nn.Softmax()

    def forward(self, x):
        embeddings = self.c(x)

        linear_activation = self.linear1(embeddings)
        non_linear_activation = self.tanh(linear_activation)

        logits = self.linear2(non_linear_activation)
        probabilities = self.softmax(logits)

        return probabilities

# Dataset 

I will try to stay close to the paper as much as I can, so we will use brown corpus from nltk, which should be the same they use in the paper, or at least **similar**.  
I don't want to drift off so we can compere the results we get to the ones in the paper, but any variations that I do from the one in the paper would probably mean different numbers which means I could've either drifted too much or implemented something incorrectly.

## Understand the dataset

In [309]:
import nltk
from nltk.corpus import brown

nltk.download("brown")
brown

[nltk_data] Downloading package brown to /Users/shadyali/nltk_data...
[nltk_data]   Package brown is already up-to-date!


<CategorizedTaggedCorpusReader in '/Users/shadyali/nltk_data/corpora/brown'>

In [310]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [311]:
len(brown.words())

1161192

In [312]:
len(set(brown.words()))  # unique words

56057

In [313]:
non_words = [word for word in set(brown.words()) if not word.isalnum()]

In [314]:
dot_index = non_words.index(".")
non_words[dot_index : dot_index + 15]

['.',
 'sea-food',
 'husband-wife',
 'herd-owner',
 "Skolovsky's",
 '7.6%',
 "Vonnegut's",
 'working-class',
 '32-degrees-F',
 '400-401',
 'south-central',
 'pistol-packing',
 'merry-go-round',
 'wine-',
 "'55"]

So we have about $56k$ unique words, which includes punctuations, words mixed with punctuations, alphanumerals, etc.  
The paper replaced all words with frequency $< 3$ with a special token of some kind. And I'm not sure if I understand what's exactly done correctly so I won't do that.

Let's do some regex for now to fix seperate things like *'herd-owner'*, etc. and then tokenize directly.

In [315]:
sentences = [
    " ".join(sent_words) for sent_words in brown.sents()
]  #   Join each sentence to be one string per sentence
sentences[:3]

["The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .",
 "The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted .",
 "The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. ."]

As you can notice, using join with " " puts a space in several places we don't want.  
So we'll use regex to solve this. **I ended up using chatGPT to get almost all patterns :(**

In [316]:
import regex as re

#   Fix the space at the sentences end before the fullstop
#   And also the space before commas.

punctuation_pattern = re.compile(r"[ ](?:,)|[ ](?:\.$)")
sentences = [
    re.sub(
        punctuation_pattern,
        lambda match: match.group().replace(" ", ""),
        sentence,
    )
    for sentence in sentences
]
print(sentences[:3])

["The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place.", "The jury further said in term-end presentments that the City Executive Committee, which had over-all charge of the election, `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted.", "The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr.."]


In [317]:
#   Lets fix quotes
quotes_pattern = re.compile(r"(?=``).*?(?<='')")

sentences = [
    re.sub(
        quotes_pattern,
        lambda match: match.group().replace("`` ", '"').replace(" ''", '"'),
        sentence,
    )
    for sentence in sentences
]
print(sentences[:3])

['The Fulton County Grand Jury said Friday an investigation of Atlanta\'s recent primary election produced "no evidence" that any irregularities took place.', 'The jury further said in term-end presentments that the City Executive Committee, which had over-all charge of the election, "deserves the praise and thanks of the City of Atlanta" for the manner in which the election was conducted.', 'The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible "irregularities" in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr..']


In [318]:
full_text = " ".join(sentences)

In [319]:
full_text[5000:5500]

"torney. Hartsfield has been mayor of Atlanta, with exception of one brief interlude, since 1937. His political career goes back to his election to city council in 1923. The mayor's present term of office expires Jan. 1. He will be succeeded by Ivan Allen Jr., who became a candidate in the Sept. 13 primary after Mayor Hartsfield announced that he would not run for reelection. Georgia Republicans are getting strong encouragement to enter a candidate in the 1962 governor's race, a top official said"

## Tokenization

I will directly use one step of regex used in [gpt-2's tokenizer](https://github.com/openai/gpt-2/blob/master/src/encoder.py), not the whole tokenizer.

In [320]:
import regex as re


#   Encoding function which splits a sequence (text) into a list
def _encode(sequence: str):
    pattern = re.compile(
        r"'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+",
        flags=re.IGNORECASE,
    )
    return re.findall(pattern=pattern, string=sequence)

You can understand what this regex pattern does from [here](https://regex101.com/r/L82kB5/1)

In [321]:
vocab = set(_encode(full_text))

In [322]:
len(vocab)

52908

In [323]:
words = [word for word in vocab]
words[20000:20020]

[' Dunes',
 ' Scrivener',
 'Tara',
 ' Product',
 ' jure',
 ' Knightfall',
 ' Towne',
 ' recitals',
 ' dentists',
 ' paymaster',
 ' timbre',
 ' endeavoring',
 ' brazenly',
 ' bout',
 ' froth',
 ' unities',
 ' hollows',
 ' route',
 ' dead',
 ' detractors']

I feel like this would be good enough to just illustrate things, so I won't try to compress the vocabulary further

We want to add a token for padding, we would need padding when the sequence we have is smaller than the context window. Or if a word is out of the vocab (we haven't encountered before)

In [324]:
word_to_token = {word: token for token, word in enumerate(vocab)}

#   Add the special padding token
word_to_token[""] = len(vocab)

In [325]:
assert max(word_to_token.values()) == len(vocab)

In [326]:
list(word_to_token.items())[:5]

[(' directrices', 0),
 (' Casino', 1),
 (' Albany', 2),
 (' triphenylarsine', 3),
 (' barnstormer', 4)]

In [327]:
#   Tokenization function
def _tokenize(words: list[str]) -> list[int]:
    tokens = [word_to_token.get(word, len(vocab)) for word in words]
    return tokens

Let's test encode and tokenize on the full text now

In [328]:
full_text_tokens = _tokenize(_encode(full_text))

In [329]:
_encode(full_text)[:15]

['The',
 ' Fulton',
 ' County',
 ' Grand',
 ' Jury',
 ' said',
 ' Friday',
 ' an',
 ' investigation',
 ' of',
 ' Atlanta',
 "'s",
 ' recent',
 ' primary',
 ' election']

In [330]:
full_text_tokens[:15]

[35378,
 10297,
 5499,
 15230,
 24146,
 48460,
 46026,
 13888,
 20731,
 45927,
 19325,
 4364,
 50112,
 29036,
 28106]

In [331]:
word_to_token["The"]

35378

In [332]:
_encode("I'am embedding things.")

['I', "'", 'am', ' embedding', ' things', '.']

In [333]:
_tokenize(_encode("I'am embedding things."))

[32874, 12369, 37549, 52908, 33490, 9421]

So there you have it! We can tokenize now

Lets structure our data for training now

## Structuring Dataset

We will do batch training, so we want to divide our corpus into batches of sequences.  
We will make a class for the dataset where it will create a list of a sliding window of $n-1$ words, and the $n^{th}$ is the label.  
The class the handle tokenization internally and will provide functions to encode and decode words to tokens and vice-versa.

In [334]:
import regex as re
from torch.utils.data import Dataset
import torch

class Corpus(Dataset):
    def __init__(self, full_text: str, n: int):
        super().__init__()
        self.context_window: int = n

        self.vocab = set(self._encode(full_text))
        self.vocab_length: int = len(vocab)

        self.word_to_token: dict[str, int] = {
            word: token for token, word in enumerate(vocab)
        }
        self.word_to_token[""] = self.vocab_length

        self.token_to_word: dict[int, str] = {
            token: word for token, word in enumerate(vocab)
        }
        self.token_to_word[self.vocab_length] = ""

        self.full_text_tokenized: list[int] = self._tokenize(self._encode(full_text))
        self.define_windows()

    # I will reuse the functions and logic from previous cells
    def _encode(self, text: str) -> list[str]:
        pattern: re.Pattern[str] = re.compile(
            r"'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+",
            flags=re.IGNORECASE,
        )
        return re.findall(pattern=pattern, string=text)

    def _tokenize(self, words: list[str]) -> list[int]:
        tokens: list[int] = [
            self.word_to_token.get(word, self.vocab_length) for word in words
        ]
        return tokens

    def tokenize_text(cls, text: str) -> torch.Tensor:
        return torch.tensor(cls._tokenize(cls._encode(text)))

    def decode(self, tokens: torch.Tensor) -> list[str]:
        tokens = tokens.tolist() if tokens.shape else [tokens.tolist()] # A scaler value will output the value without list, raising an error below
        
        words = [self.token_to_word[token] for token in tokens]
        return words

    def define_windows(self) -> None:
        self.sequences = []
        self.next_token = []

        text_length = len(self.full_text_tokenized)
        for index in range(text_length):
            if index + self.context_window - 1 < text_length:
                self.sequences.append(
                    self.full_text_tokenized[index : index + self.context_window - 1]
                )
                self.next_token.append(
                    self.full_text_tokenized[index + self.context_window - 1]
                )

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, window):
        return torch.tensor(self.sequences[window]), torch.tensor(self.next_token[window])

In [335]:
dataset = Corpus(full_text=full_text, n=4)

In [336]:
dataset.decode(dataset[60][0]), dataset.decode(dataset[60][1])

([' of', ' Atlanta', '"'], [' for'])

We can now use the DataLoader class to get an iterator to produce batches of training data for us

In [337]:
from torch.utils.data import DataLoader

data_iterator = iter(DataLoader(
    dataset=dataset, batch_size=50, shuffle=True
))

In [338]:
input, label = next(data_iterator)
print(f"Feature batch shape: {input.shape}")
print(f"Labels batch shape: {label.shape}")

Feature batch shape: torch.Size([50, 3])
Labels batch shape: torch.Size([50])


The loader is working!!

I think we are ready now to split our data, ***AND TRAIN OUR MODEL***

# MODEL TRAINING